In [98]:
import pandas as pd
import nltk.data
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import gensim
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Dropout, Flatten
from keras.layers import Activation
import numpy as np
import chardet
import logging
from keras.models import Model
from keras.layers import Input,Dense,Dropout,Flatten
from keras.layers import Conv1D,MaxPooling1D,AveragePooling1D
from keras.layers import LSTM,Bidirectional
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, roc_auc_score, precision_score
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn import cross_validation
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from nltk.stem import PorterStemmer


In [100]:
def clean_text(dataframe,stop):
#Cleaning text    
    dataframe["doc"] = dataframe['text']
#Remove numbers
    dataframe.loc[:,"doc"] = dataframe.doc.str.replace(r'\d+','')
#Remove floats
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).replace(r'(\d*\.?\d*)','')
#Remove Punctuation
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x : " ".join(re.findall('[\w]+',x)))
#Convert to lower text
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))      
#Stop words removal
    if stop == True:
        dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].astype(str)
        stop = stopwords.words('english')
        dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(lambda x: " ".join(x.lower() for x in str(x).split() if x not in stop))
#Reomve top 50 common words
    freq = pd.Series(' '.join(dataframe['doc']).split()).value_counts()[:50]
    freq = list(freq.index)
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Stemming        
    st = PorterStemmer()
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Moving cleaned text to another column
    dataframe['cleaned_text'] = dataframe["doc"]
#Tokenization                               
    dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(nltk.word_tokenize)
    
    return dataframe



In [129]:
def clean_text1(dataframe):
#Cleaning text    
    dataframe["doc"] = dataframe['text']
#Remove numbers
    dataframe.loc[:,"doc"] = dataframe.doc.str.replace(r'\d+','')
#Remove floats
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).replace(r'(\d*\.?\d*)','')
#Remove Punctuation
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x : " ".join(re.findall('[\w]+',x)))
#Convert to lower text
    dataframe.loc[:,"doc"] = dataframe.doc.astype(str).apply(lambda x: " ".join(x.lower() for x in x.split()))      
#Stop words removal
    stop = stopwords.words('english')
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#Reomve top 50 common words
    freq = pd.Series(' '.join(dataframe['doc']).split()).value_counts()[:50]
    freq = list(freq.index)
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
#Stemming        
    st = PorterStemmer()
    dataframe.loc[:,'doc'] = dataframe.loc[:,'doc'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

# Moving cleaned text to another column
    dataframe['cleaned_text'] = dataframe["doc"]
#Tokenization                               
    dataframe.loc[:,"doc"] = dataframe.loc[:,"doc"].apply(nltk.word_tokenize)
    
    return dataframe

In [4]:
def w2c_model(doc,size,window,min_count,workers,iter,sg,length,numepoch):    
    model = gensim.models.Word2Vec(size=size, window=window, min_count=min_count, workers=workers,iter=iter,sg=sg)
    model.build_vocab(doc)
    model.train(doc,total_examples=length,epochs = numepoch)
    return model

In [5]:
#Convert clean text into sequence base on word2vec
def text2seq(doc, w2v, seq_len, emb_size):
    padding = np.array([0 for __ in range(emb_size)])
    seq = []
    for token in doc[: seq_len]:
        try:
            seq.append(w2v.wv[token])
        except:
            aa = 1

    seq += [padding for __ in range(seq_len - len(seq))]
    #print(len(seq))
    return seq

In [6]:
def one_hot(series):
    values=sorted(list(set(series)))
    return np.vstack(map(lambda x: [x==v for v in values],series))

In [7]:
def prepro(dataframe, w2v,seq_len, emb_size, target_col):
    X = dataframe['doc'].apply(lambda doc: text2seq(doc, w2v, seq_len, emb_size))
    X = np.vstack(X).reshape(len(dataframe), seq_len, emb_size)
    y = one_hot(dataframe[target_col])
    return X, y

In [37]:
# #Model building 
# def model1(seq_len, emb_size, num_labels):
#     input_layer = Input(shape=(seq_len, emb_size))
#     x = Conv1D(32, 3, activation='relu')(input_layer)
#     x = Flatten()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(64, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)
#     output_layer = Dense(num_labels, activation='softmax')(x)
#     model = Model(input_layer, output_layer)
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
#     return model

In [70]:
def model2():
    input_layer = Input(shape=(100, 100))
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(9, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [93]:
# def define_model():
#     model = Sequential()
#     model.add(Embedding(882, 100))
#     model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
#     model.add(MaxPooling1D(pool_size=2))
#     model.add(Flatten())
#     model.add(Dense(10, activation='relu'))
#     model.add(Dense(2, activation='sigmoid'))
#     # compile network
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     # summarize defined model
#     #model.summary()
#     #plot_model(model, to_file='model.png', show_shapes=True)
#     return model

In [71]:
def build_model():
    input_layer = Input(shape=(100, 100))
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)  
    x = LSTM(100, dropout=0.2, recurrent_dropout=0.2)(x) 
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(9, activation='softmax')(x)
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [144]:
# You can change any numbers except those two I highlighted, or add any dense layers or change activations, 
# dropout rate


In [125]:
def build_model2():
    input_layer = Input(shape=(100,100)) #These two numbers cannot change
    x = Conv1D(32, 3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)  
    x = LSTM(100, return_sequences = True,dropout=0.2,recurrent_dropout=0.05)(input_layer)
    x = Flatten()(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.2)(x)
    x = Dense(64, activation='relu')(x)
#     x = Dropout(0.2)(x)
    x = Dense(128, activation='relu')(x)
#     x = Dropout(0.2)(x)
    output_layer = Dense(9, activation='softmax')(x) # 9 is the number represent how many Labels we have 0-8
    model = Model(input_layer, output_layer)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    return model

In [10]:
def fit_model(X,y,weightsfile,num_labels,seq_len,emb_size,epochs):
    checkpoint = ModelCheckpoint(weightsfile, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    model_dl = model1(seq_len, emb_size, num_labels)
    model_dl.fit(X, y, epochs=epochs, batch_size=16,callbacks=callbacks_list, validation_split=0.2)
    return model_dl

In [11]:
df = pd.read_csv('data.csv')

In [12]:
df.head()

,Unnamed: 0,ID,text,Label
0,0,20030006121,Passive radio frequency identification system ...,8
1,1,20030009666,Methods and apparatus for efficient computatio...,0
2,2,20030014337,"Systems, methods and computer program products...",8
3,3,20030019942,System and method for electronically readable ...,8
4,4,20030033242,System and method for automated process of dea...,6


In [130]:
#Clean the text column
df = clean_text1(df)
#Assign processed column to be the target column for the word2vec
doc = df.doc
#create a word2vec model to use as embedding layers in CNN
w2c = w2c_model(doc,size=100,window=5,min_count=1,workers=6,iter=10,sg=0,length=len(doc),numepoch=10)

In [134]:
df.head()

,Unnamed: 0,ID,text,Label,doc,cleaned_text
0,0,20030006121,Passive radio frequency identification system ...,8,"[passiv, radio, frequenc, identif, identifi, t...",passiv radio frequenc identif identifi track c...
1,1,20030009666,Methods and apparatus for efficient computatio...,0,"[method, apparatu, effici, comput, way, chain,...",method apparatu effici comput way chain crypto...
2,2,20030014337,"Systems, methods and computer program products...",8,"[system, method, program, product, perform, ge...",system method program product perform gener co...
3,3,20030019942,System and method for electronically readable ...,8,"[electron, readabl, power, sourc, improv, cont...",electron readabl power sourc improv contain el...
4,4,20030033242,System and method for automated process of dea...,6,"[autom, process, deal, structur, autom, deal, ...",autom process deal structur autom deal custom ...


In [142]:
df = df.sample(frac=1)

In [131]:
#Create sequence and embedding layer base on word2vec 
X, y = prepro(df,w2c,seq_len=100, emb_size=100,target_col='Label')

In [132]:
# Wrap Keras model so it can be used by scikit-learn
CNN = KerasClassifier(build_fn=build_model2, 
                      batch_size=100,
                            epochs=10)
                                 

In [122]:
X.shape

(882, 100, 100)

In [97]:
type(X)

numpy.ndarray

In [127]:
#Use cross validation predict to make prediction on the entire dataset
cnn_pred = cross_val_predict(CNN, X, df.Label, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(df.Label,cnn_pred))

Epoch 1/10
793/793 [==============================] - 17s 21ms/step - loss: 1.5652 - acc: 0.4338
Epoch 2/10
793/793 [==============================] - 3s 3ms/step - loss: 0.7713 - acc: 0.7251
Epoch 3/10
793/793 [==============================] - 3s 4ms/step - loss: 0.3681 - acc: 0.8663
Epoch 4/10
793/793 [==============================] - 3s 4ms/step - loss: 0.1855 - acc: 0.9571
Epoch 5/10
793/793 [==============================] - 3s 3ms/step - loss: 0.1069 - acc: 0.9786
Epoch 6/10
793/793 [==============================] - 3s 4ms/step - loss: 0.0472 - acc: 0.9924
Epoch 7/10
793/793 [==============================] - 3s 3ms/step - loss: 0.0519 - acc: 0.9899
Epoch 8/10
793/793 [==============================] - 3s 3ms/step - loss: 0.0538 - acc: 0.9912
Epoch 9/10
793/793 [==============================] - 3s 4ms/step - loss: 0.0450 - acc: 0.9861
Epoch 10/10
793/793 [==============================] - 3s 3ms/step - loss: 0.0423 - acc: 0.9861
Epoch 1/10
793/793 [===========================

Epoch 7/10
794/794 [==============================] - 3s 4ms/step - loss: 0.0548 - acc: 0.9874
Epoch 8/10
794/794 [==============================] - 3s 4ms/step - loss: 0.0493 - acc: 0.9887
Epoch 9/10
794/794 [==============================] - 3s 4ms/step - loss: 0.0423 - acc: 0.9924
Epoch 10/10
794/794 [==============================] - 3s 4ms/step - loss: 0.0267 - acc: 0.9950
Epoch 1/10
794/794 [==============================] - 20s 25ms/step - loss: 1.5355 - acc: 0.4962
Epoch 2/10
794/794 [==============================] - 3s 4ms/step - loss: 0.7501 - acc: 0.7431
Epoch 3/10
794/794 [==============================] - 3s 4ms/step - loss: 0.3342 - acc: 0.8879
Epoch 4/10
794/794 [==============================] - 3s 3ms/step - loss: 0.1488 - acc: 0.9610
Epoch 5/10
794/794 [==============================] - 3s 3ms/step - loss: 0.0809 - acc: 0.9824
Epoch 6/10
794/794 [==============================] - 3s 3ms/step - loss: 0.0825 - acc: 0.9761
Epoch 7/10
794/794 [===========================

In [133]:
#Accuracy score
accuracy_score(df.Label, cnn_pred)*100

55.21541950113379

In [94]:
# CNN1 = KerasClassifier(build_fn=define_model, 
#                             epochs=20)

In [40]:
# cross_val_score(CNN, X, y, cv=10)

In [135]:
# scoring = {'acc': 'accuracy',
#            'precision': 'precision_macro',
#            'recall': 'recall_macro',
#            'f1_score' : 'f1_macro'}
# cnn_scores = cross_validate(CNN, X, df.Label, scoring=scoring,
#                          cv=10, return_train_score=True)

In [136]:
# print(cnn_scores.keys())
# print(cnn_scores['test_acc'].mean())
# print(cnn_scores['test_precision'].mean())
# print(cnn_scores['test_recall'].mean())
# print(cnn_scores['test_f1_score'].mean())

In [137]:
# #Use cross validation predict to make prediction on the entire dataset
# cnn_pred = cross_val_predict(CNN, X, df.Label, cv=10)

# #Use classification report to get the precision & recall table, compare with prediction and actual labels
# print(classification_report(df.Label,cnn_pred))

In [138]:
# accuracy_score(df.Label, cnn_pred)*100

In [29]:
df.groupby('Label').count()

,Unnamed: 0,ID,text,doc,cleaned_text
Label,,,,,
0,274,274,274,274,274
1,84,84,84,84,84
2,77,77,77,77,77
3,61,61,61,61,61
4,37,37,37,37,37
5,42,42,42,42,42
6,45,45,45,45,45
7,34,34,34,34,34
8,228,228,228,228,228


In [49]:
#Create new dataframe that only contains label 0 & 8, both contains 74 labels each
d1 = df[df['Label']==0].sample(74)
d2 = df[df['Label']==8].sample(74)
#Append them into a new dataframe
d3 = d1.append(d2, ignore_index=True)

#Create new dataframe that only contains label 1 to 7
d4 = df[(df['Label'] != 0 ) & (df['Label'] != 8)]
#Append them together
ds = d4.append(d3, ignore_index=True)

In [80]:
ds = ds.sample(frac=1)

In [81]:
X1, y = prepro(ds,w2c,seq_len=100, emb_size=100,target_col='Label')

In [82]:
#Use cross validation predict to make prediction on the entire dataset
CNN_pred = cross_val_predict(CNN, X1, ds.Label, cv=10)

#Use classification report to get the precision & recall table, compare with prediction and actual labels
print(classification_report(ds.Label,CNN_pred))

Epoch 1/20
475/475 [==============================] - 7s 14ms/step - loss: 1.6561 - acc: 0.4021
Epoch 2/20
475/475 [==============================] - 1s 3ms/step - loss: 0.6959 - acc: 0.7874
Epoch 3/20
475/475 [==============================] - 1s 3ms/step - loss: 0.3426 - acc: 0.9179
Epoch 4/20
475/475 [==============================] - 1s 3ms/step - loss: 0.1911 - acc: 0.9642
Epoch 5/20
475/475 [==============================] - 2s 3ms/step - loss: 0.1152 - acc: 0.9832
Epoch 6/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0855 - acc: 0.9853
Epoch 7/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0664 - acc: 0.9937
Epoch 8/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0544 - acc: 0.9853
Epoch 9/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0653 - acc: 0.9853
Epoch 10/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0583 - acc: 0.9874
Epoch 11/20
475/475 [===========================

475/475 [==============================] - 2s 3ms/step - loss: 0.0865 - acc: 0.9853
Epoch 7/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0780 - acc: 0.9853
Epoch 8/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0809 - acc: 0.9874
Epoch 9/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0567 - acc: 0.9874
Epoch 10/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0525 - acc: 0.9874
Epoch 11/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0476 - acc: 0.9895
Epoch 12/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0593 - acc: 0.9895
Epoch 13/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0510 - acc: 0.9895
Epoch 14/20
475/475 [==============================] - 1s 3ms/step - loss: 0.0581 - acc: 0.9853
Epoch 15/20
475/475 [==============================] - 2s 3ms/step - loss: 0.0401 - acc: 0.9874
Epoch 16/20
475/475 [==============================] - 

476/476 [==============================] - 1s 3ms/step - loss: 0.0576 - acc: 0.9874
Epoch 12/20
476/476 [==============================] - 1s 3ms/step - loss: 0.0628 - acc: 0.9874
Epoch 13/20
476/476 [==============================] - 2s 3ms/step - loss: 0.0407 - acc: 0.9916
Epoch 14/20
476/476 [==============================] - 2s 3ms/step - loss: 0.0642 - acc: 0.9895
Epoch 15/20
476/476 [==============================] - 1s 3ms/step - loss: 0.0572 - acc: 0.9874
Epoch 16/20
476/476 [==============================] - 2s 3ms/step - loss: 0.0553 - acc: 0.9874
Epoch 17/20
476/476 [==============================] - 2s 4ms/step - loss: 0.0593 - acc: 0.9895
Epoch 18/20
476/476 [==============================] - 2s 3ms/step - loss: 0.0480 - acc: 0.9895
Epoch 19/20
476/476 [==============================] - 1s 3ms/step - loss: 0.0625 - acc: 0.9916
Epoch 20/20
476/476 [==============================] - 1s 3ms/step - loss: 0.0714 - acc: 0.9853
Epoch 1/20
476/476 [==============================] 

In [83]:
accuracy_score(ds.Label, cnn_pred)*100

64.96212121212122

In [139]:
# scoring = {'acc': 'accuracy',
#            'precision': 'precision_macro',
#            'recall': 'recall_macro',
#            'f1_score' : 'f1_macro'}
# cnn_scores = cross_validate(CNN, X1, ds.Label, scoring=scoring,
#                          cv=10, return_train_score=True)

In [140]:
# print(cnn_scores.keys())
# print(cnn_scores['test_acc'].mean())
# print(cnn_scores['test_precision'].mean())
# print(cnn_scores['test_recall'].mean())
# print(cnn_scores['test_f1_score'].mean())

In [67]:
CNN_model= fit_model(X=X,y=y,weightsfile = "first.hdf5",num_labels=9,seq_len=100,emb_size=100,epochs=20)


Train on 705 samples, validate on 177 samples
Epoch 1/20
705/705 [==============================] - 2s 3ms/step - loss: 2.1314 - acc: 0.2709 - val_loss: 1.8444 - val_acc: 0.3390

Epoch 00001: val_acc improved from -inf to 0.33898, saving model to first.hdf5
Epoch 2/20
705/705 [==============================] - 1s 878us/step - loss: 1.7256 - acc: 0.3844 - val_loss: 1.6324 - val_acc: 0.4237

Epoch 00002: val_acc improved from 0.33898 to 0.42373, saving model to first.hdf5
Epoch 3/20
705/705 [==============================] - 1s 913us/step - loss: 1.3070 - acc: 0.5532 - val_loss: 1.5219 - val_acc: 0.4802

Epoch 00003: val_acc improved from 0.42373 to 0.48023, saving model to first.hdf5
Epoch 4/20
705/705 [==============================] - 1s 894us/step - loss: 0.8908 - acc: 0.6950 - val_loss: 1.4335 - val_acc: 0.4746

Epoch 00004: val_acc did not improve from 0.48023
Epoch 5/20
705/705 [==============================] - 1s 951us/step - loss: 0.5521 - acc: 0.8227 - val_loss: 1.3064 - val_a

In [141]:
# CNN_model.load_weights('first.hdf5')

In [68]:
# model_dl = build_model(seq_len=100, emb_size=100, num_labels=9)
# CNN = model_dl.fit(X, y, epochs=20, batch_size=16, validation_split=0.2)

In [73]:
# pred = cross_val_predict(CNN_model.load_weights('first.hdf5'), X, y, cv=10)
